## Kodierung Beobachtungen

In [1]:
# HIDDEN
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR); # logging.FATAL

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Überprüfung: Was ist eine Policy?

- Im RL versuchen wir, eine Policy zu lernen, was ist das nochmal genau?
- Eine Policy bildet **Beobachtungen** auf **Aktionen** ab.
- Mit anderen Worten: Die Beobachtungen sind alles, was die Policy "sieht".

#### Zufällige Seenpolicy

- Was sind die Beobachtungen im Zufallssee?
- Es ist der Standort des Spielers, dargestellt als ganze Zahl von 0 bis 15  
- Zur Auffrischung von Modul 1: Eine deterministische Policy könnte wie folgt aussehen:

| Beobachtung | Aktion |
|------|-------|
| 0 | 0 |
| 1 | 3 |
| 2 | 1 |
| 3 | 1 |
| ... | ... |
| 14 | 2 |
| 15 | 2 |

#### Zufällige Seenpolicy

Und eine nicht-deterministische Policy könnte so aussehen:

| Beobachtung | P(links) | P(unten) | P(rechts) | P(oben) | 
|------------|-------|-----------|---------|-------|
| 0 | 0 | 0.9 | 0.01 | 0.04 | 0.05
| 1 | 3 | 0.05 | 0.05 | 0.05 | 0.85
| ... | ... | ... | ...      | ...      | ...
| 15 | 2 | 0.0 | 0.0 | 0.99 | 0.01

Das bedeutet übrigens nicht, dass RLlib eine solche Tabelle lernt, aber wir können uns diese Tabelle begrifflich vorstellen.

#### Zufällige Seenpolicy

- Im Random Lake muss unsere gesamte Entscheidung auf der Position des Spielers basieren.
- Manchmal reicht das schon aus: Von Position 11 aus solltest du nach unten gehen.

```
 0 1 2 3
 4 5 6 7
 8 9 10 11
12 13 14 15
```

- Aber was ist mit Position 5, was solltest du von dort aus tun?
- Antwort: _Es kommt darauf an_. Wenn es an Position 9 ein Loch gibt, willst du nicht nach unten gehen. Das Gleiche gilt für die 6 
- Wie kann ich das entscheiden, _ohne zu wissen, wo die Löcher sind_?

#### State vs. Beobachtung, eine Zusammenfassung

- In Modul 1 haben wir den Zustand informell als alles über die Environment definiert.
- In diesem Fall wären das der Standort des Spielers und die Löcher.
- Die Beobachtung hingegen kodiert nur einen Teil des Zustands: in diesem Fall den Standort des Spielers.

#### Beobachtung = Zustand? Problem 1.

- Okay, warum dann nicht einfach die Beobachtung auf den Zustand setzen? 
- Hier gibt es zwei Probleme.
- Problem 1: Wenn das RL-System eingesetzt wird, kennst du vielleicht nicht den gesamten Status.
  - Beispiel: In einem Empfehlungssystem hat der Agent (Empfehlungsgeber) keinen Zugriff auf die Stimmung des Nutzers (ein Teil des Zustands, der das Ergebnis beeinflusst)
  - Beim überwachten Lernen wollen wir nicht auf Merkmale trainieren, auf die wir im Einsatz keinen Zugriff haben
    - Auch hier muss die Beobachtung etwas sein, auf das wir im Einsatz zugreifen können.

#### Beobachtung = Zustand? Problem 2.

- Problem 2: Es kann schwierig sein, von einer wirklich komplexen Beobachtung zu verallgemeinern.
  - Es gibt Hunderttausende von möglichen Zuständen in diesem kleinen 4x4 Random Lake Spiel.
  - Zu viele Informationen könnten für den Agenten verwirrend sein oder es könnten unangemessen viele Daten (Simulationen) benötigt werden, um sie zu verstehen.

#### Beobachtungen zur Kodierung

- Ein Teil unserer Aufgabe als RL-Praktiker ist es, eine Darstellung (oder Kodierung) für die Beobachtung zu wählen.
- Finde aus den Informationen, die der Spieler wissen darf, eine sinnvolle Darstellung dessen, was der Spieler wissen muss.
- In unserem Fall probieren wir einen Ansatz aus: Der Spieler kann "sehen", ob die 4 angrenzenden Felder Löcher sind oder nicht.
- Wir kodieren dies als 4 Binärzahlen.

#### Beobachtungen zur Kodierung

```
.OO.
....
O.P.
...G
```

- In dieser Situation gibt es keine Löcher um den Spieler herum, also "sieht" der Spieler `[0 0 0 0]` 
- Mit anderen Worten: Die Beobachtung ist hier `[0 0 0 0]`.

#### Beobachtungen zur Kodierung

```
.OO.
..P.
O.O.
...G
```

- Hier "sieht" der Spieler Löcher nach oben und unten, also ist die Beobachtung `[0 1 0 1]` (links, unten, rechts, oben)

#### Beobachtungen zur Kodierung

Was ist mit Kanten?

```
....
..OP
O.OO
...G
```

- Dies ist unsere Wahl, wenn wir den Beobachtungsraum gestalten.
- Ich entscheide mich dafür, "abseits des Rasters" als Löcher darzustellen, d.h. wir tun so, als ob der See so aussieht:
 
```
OOOOOO
O....O
O..OPO
OO.OOO
O...GO
OOOOOO
```

- Hier sieht der Spieler Löcher links, unten und rechts, also lautet die Beobachtung `[1 1 1 0]` (links, unten, rechts, oben)
- Es gibt aber vielleicht bessere Ansätze, denn in ein Loch zu fallen ist schlimmer (die Episode endet) als über den Rand zu laufen (es passiert nichts).

#### Kodierung unserer Beobachtungen

- Jetzt, wo wir einen Plan haben, wie ändern wir den Code?
- Da wir unsere Klasse so strukturiert haben, dass sie eine Methode "Beobachtung" hat, müssen wir nur diese ändern:

In [2]:
from envs_03 import RandomLake

class RandomLakeObs(RandomLake):
    def observation(self):
        i, j = self.player

        obs = []
        obs.append(1 if j==0 else self.holes[i,j-1]) # left
        obs.append(1 if i==3 else self.holes[i+1,j]) # down
        obs.append(1 if j==3 else self.holes[i,j+1]) # right
        obs.append(1 if i==0 else self.holes[i-1,j]) # up
        
        obs = np.array(obs, dtype=int) # cast to numpy array (optional)
        return obs

- Der Code erstellt eine Variable `obs`, in der jeder Eintrag 1 ist, wenn diese Richtung von der Kante wegführt **oder** dort ein Loch vorhanden ist.

In [3]:
# HIDDEN
import gym

#### Kodierung unserer Beobachtungen

- Es ist noch eine weitere Codeänderung erforderlich, und zwar der Konstruktor, in dem der Beobachtungsraum definiert wird.
- Unsere Beobachtungen waren bisher eine Ganzzahl von 0 bis 15, also haben wir

In [4]:
observation_space = gym.spaces.Discrete(16)

Das gilt auch für Aktionen:

In [5]:
action_space = gym.spaces.Discrete(4)      

- Allerdings sind unsere Beobachtungen jetzt Arrays aus 4 Zahlen und nicht mehr eine einzelne Zahl.
- Um dies zu verdeutlichen, verwenden wir "gym.spaces.MultiDiscrete" anstelle von "gym.spaces.Discrete".
- Multi, weil wir mehrere Zahlen haben, aber immer noch diskret, weil jede der 4 Zahlen nur 2 mögliche Werte annehmen kann (0 oder 1).
- Hier ist der Code:

In [6]:
class RandomLakeObs(RandomLake):
    def __init__(self, env_config=None):
        self.observation_space = gym.spaces.MultiDiscrete([2,2,2,2])
        self.action_space = gym.spaces.Discrete(4)      

(Beachte, dass `gym` auch einen `MultiBinary`-Raumtyp hat, aber dieser wird derzeit nicht von RLlib unterstützt)

#### Teste unsere neue Environment

Testen wir es aus!

In [7]:
# HIDDEN
import numpy as np
np.random.seed(42)

In [8]:
from envs_03 import RandomLakeObs

env = RandomLakeObs()
env.reset()

array([1, 1, 0, 1])

In [9]:
env.render()

🧑🧊🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️


Hier sehen wir die erwartete Beobachtung, die "Löcher" nach links, unten und oben anzeigt.

Anmerkungen 

Links und oben sind die Kartenränder, und unten ist ein tatsächliches Loch.

#### Teste unsere neue Environment

Lass uns versuchen, nach rechts zu gehen:

In [10]:
env.step(2)

(array([0, 1, 0, 1]), 0, False, {'player': (0, 1), 'goal': (3, 3)})

In [11]:
env.render()

🧊🧑🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️


Jetzt sehen wir Löcher in der Abwärts- und Aufwärtsrichtung, wie erwartet.

#### Training mit unseren neuen Beobachtungen

- Unsere neuen Beobachtungen scheinen zu funktionieren, aber helfen sie dem Agenten auch beim Lernen?
- Erinnere dich daran, dass wir mit unserem `Diskreten(16)` Beobachtungsraum nicht viel mehr als eine Erfolgsquote von 30% erreichen konnten.
- Versuchen wir es noch einmal:

In [12]:
# HIDDEN
from utils_03 import lake_default_config

In [13]:
ppo = lake_default_config.build(env=RandomLakeObs)

for i in range(8):
    ppo.train()

In [14]:
ppo.evaluate()["evaluation"]["episode_reward_mean"]

0.6420454545454546

- Das ist viel besser als die ~30%, die wir vorher bekommen haben!
- Das macht Sinn... unser Agent kann die Löcher jetzt "sehen", anstatt blindlings zu laufen.

#### Lass uns das Gelernte anwenden!

## Analogie zum überwachten Lernen: Beobachtungsraum
<!-- multiple choice -->

In den Folien haben wir den Beobachtungsraum für unseren Agenten verändert und dadurch eine höhere Belohnung erzielt. Welchem Aspekt des überwachten Lernens ähnelt das am meisten?

- [x] Feature Engineering | Du hast es erfasst! Unser Beobachtungsraum dient als Merkmalsraum, auf den unsere Policy einwirken soll.
- [Modellauswahl | Nicht ganz. Aber wie wir sehen werden, gibt es auch im RL einen Platz für die Modellauswahl!
- [Hyperparameter-Abstimmung | Nicht ganz. Aber wie wir noch sehen werden, gibt es auch im RL einen Platz für Hyperparameter-Tuning!
- [Auswählen einer Verlustfunktion

## Einschließlich der Position des Spielers
<!-- multiple choice -->

In unserer neuen Beobachtungsdarstellung haben wir den Standort des Spielers aus der Beobachtung _entfernt_ und _nur_ das Vorhandensein der Löcher in der Nähe berücksichtigt. Wenn wir einen Beobachtungsraum brauchen, der sowohl die Wände in der Nähe als auch den Standort des Spielers enthält, welchen der folgenden Turnhallenräume könnten wir dann verwenden?

- [ ] `gym.spaces.Discrete(5)` | Versuch es noch einmal!
- [x] `gym.spaces.MultiDiscrete([2,2,2,2,2,16])` | Ja! Die ersten 4 Zahlen stehen für die Löcher und die letzte Zahl für den Standort des Spielers.
- [ ] `gym.spaces.MultiDiscrete([2,2,2,2]) + gym.spaces.Discrete(16)` | Versuche es noch einmal; leider können wir keine Turnhallenplätze hinzufügen.
- [ ] `gym.spaces.MultiDiscrete([32,32,32,32])` | Das könnte funktionieren, ist aber eine verwirrende/redundante Darstellung.

## Behandlung der Kanten
<!-- multiple choice -->

In den Folien haben wir uns entschieden, Kanten wie Löcher zu behandeln. Erinnere dich an dieses Bild:

```
OOOOOO
O....O
O..OPO
OO.OOO
O...GO
OOOOOO
```

Kanten und Löcher unterscheiden sich jedoch voneinander: Wenn du in eine Kante läufst, bewirkt das nichts, während du in ein Loch läufst, dass die Episode zu Ende ist. Das könnte ein wichtiger Unterschied sein, besonders in einer "schlüpfrigen" Version der Environment, in der die Ergebnisse von Aktionen nicht deterministisch sind 

Um dieses Problem zu lösen, beschließen wir, den Beobachtungsraum zu ändern. Der Agent "sieht" immer noch nur die vier Quadrate um ihn herum, aber jetzt sieht er, ob jedes Quadrat ein leeres Feld, ein Loch oder eine Kante ist. Welchen der folgenden Turnhallen-Beobachtungsräume könnten wir für diese Darstellung verwenden?

- [ ] `gym.spaces.MultiDiscrete([2,2,2,2,2,2,2,2,2])` | Versuch es noch einmal. Denke daran, dass der Agent immer noch nur 4 Quadrate "sieht".
- [gym.spaces.MultiDiscrete([3,3,3,3,3,3,3,3,3])` | Versuche es noch einmal!
- [ ] `gym.spaces.MultiDiscrete([2,2,2,2])` | Dies ist dasselbe wie das vorherige Feld, aber wir haben eine Änderung vorgenommen.
- [x] `gym.spaces.MultiDiscrete([3,3,3,3])` | Du hast es geschafft! Es gibt jetzt 3 mögliche Optionen für das, was der Agent an jedem Platz "sehen" kann.

In [15]:
# TODO / note to self
# query_policy(trainer, RandomLakeObs(), [1,1,1,1])
# shows that it wants to go up. this is because the above "hole" is probably an edge based on its learning. fascinating.

## Implementierung der Kanten
<!-- coding exercise -->

Der folgende Code zeigt die Funktion `Beobachtung` für den aktuellen Beobachtungsraum. Ändere den Code so, dass er den neuen Beobachtungsraum verwendet, wobei 0 für einen leeren Raum, 1 für ein Loch und 2 für eine Kante steht 

In [16]:
# EXERCISE

from envs_03 import RandomLake

class RandomLakeObs2(RandomLakeObs):
    def observation(self):
        i, j = self.player

        obs = []
        obs.append(1 if j==0 else self.holes[i,j-1]) # left
        obs.append(1 if i==3 else self.holes[i+1,j]) # down
        obs.append(1 if j==3 else self.holes[i,j+1]) # right
        obs.append(1 if i==0 else self.holes[i-1,j]) # up
        
        obs = np.array(obs, dtype=int) # cast to numpy array
        return obs

np.random.seed(42)
env = RandomLakeObs2()
obs = env.reset()
env.render()
print(obs)

🧑🧊🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️
[1 1 0 1]


In [17]:
# SOLUTION

from envs_03 import RandomLake

class RandomLakeObs2(RandomLakeObs):
    def observation(self):
        i, j = self.player

        obs = []
        obs.append(2 if j==0 else self.holes[i,j-1]) # left
        obs.append(2 if i==3 else self.holes[i+1,j]) # down
        obs.append(2 if j==3 else self.holes[i,j+1]) # right
        obs.append(2 if i==0 else self.holes[i-1,j]) # up
        
        obs = np.array(obs, dtype=int) # cast to numpy array
        return obs

np.random.seed(42)
env = RandomLakeObs2()
obs = env.reset()
env.render()
print(obs)

🧑🧊🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️
[2 1 0 2]


## Was der Agent sieht
<!-- coding exercise -->

Mit unserer neuen Kodierung des Beobachtungsraums "sieht" der Agent nur die 4 Räume um ihn herum und hat nur diese Informationen zur Verfügung, um seine Entscheidungen zu treffen. Die Codezelle unten zeigt, was der Agent "sieht", während er auf dem Zufallssee navigiert. Du kannst Aktionen mit der Tastatur eingeben, indem du die Wörter "links", "unten", "rechts" oder "oben" (oder kurz "l", "d", "r", "u") eintippst und die Simulation wird dir das Ergebnis zeigen. (Tippe "quit", um zu beenden.) Spiele das Spiel, bis du das Ziel erreicht hast. Versuche währenddessen, den See zu kartografieren (vielleicht durch Zeichnen auf einem Blatt Papier).

In [18]:
# TODO / NOTE:
# THIS EXERCISE DOES NOT HAVE A "solution"
# the code is here ONLY to help them answer the multiple choice

In [ ]:
# EXERCISE

import numpy as np
from envs_03 import RandomLakeObs

actions = {"left" : 0, "down" : 1, "right" : 2, "up" : 3, 
           "l" : 0, "d" : 1, "r" : 2, "u" : 3}

np.random.seed(45)
env = RandomLakeObs()
obs = env.reset()

act = "start"
done = False

while not done:
   
    obs_print = [['.']*3 for i in range(3)]
    obs_print[1][1] = "P"
    if obs[0]:
        obs_print[1][0] = "O"
    if obs[1]:
        obs_print[2][1] = "O"
    if obs[2]:
        obs_print[1][2] = "O"
    if obs[3]:
        obs_print[0][1] = "O"
    print("Observation:")
    print("\n".join(list(map(lambda c: "".join(c), obs_print))))
    print()
    
    while act != "quit" and act not in actions: 
        act = input() # gather keyboard input 
    
    if act == "quit":
        break
        
    obs, rew, done, _ = env.step(act)
    
if done:
    if rew > 0:
        print("You win! +1 reward 🎉")
    else:
        print("You fell into the lake 😢")

Observation:
.O.
OP.
...



In [ ]:
# SOLUTION

import numpy as np
from envs_03 import RandomLakeObs

actions = {"left" : 0, "down" : 1, "right" : 2, "up" : 3, 
           "l" : 0, "d" : 1, "r" : 2, "u" : 3}

np.random.seed(45)
env = RandomLakeObs()
obs = env.reset()

act = "start"
done = False

while not done:
   
    obs_print = [['.']*3 for i in range(3)]
    obs_print[1][1] = "P"
    if obs[0]:
        obs_print[1][0] = "O"
    if obs[1]:
        obs_print[2][1] = "O"
    if obs[2]:
        obs_print[1][2] = "O"
    if obs[3]:
        obs_print[0][1] = "O"
    print("Observation:")
    print("\n".join(list(map(lambda c: "".join(c), obs_print))))
    print()
    
    while act != "quit" and act not in actions: 
        act = input() # gather keyboard input 
    
    if act == "quit":
        break
        
    obs, rew, done, _ = env.step(act)
    
if done:
    if rew > 0:
        print("You win! +1 reward 🎉")
    else:
        print("You fell into the lake 😢")

#### Wie sieht der See aus?

Welche Karte des Sees in der obigen Frage ist nach deinen Erkundungen die richtige?

```
 (A) (B) (C) (D)
P..O P.OO P..O P.OO
..OO .OOO ..OO .OOO
O...     O...     O...     O..O
...G ...G ..OG ...G
```

- [x] (A)
- [ ] (B)
- [ ] (C)
- [ ] (D)

In [ ]:
# TODO
# could also considering showing a BAD environment encoding to contrast with this reasonable one, as in the next slide deck!